<a href="https://colab.research.google.com/github/Soflem0715/img/blob/main/Ex_1_Lemecheva_Sophia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Установка необходимых библиотек, если они не установлены (в Colab обычно предустановлены, но на всякий случай)
!pip install pandas numpy statsmodels scipy scikit-learn -q  # -q для тихой установки, чтобы не засорять вывод

# Импорт библиотек для работы с данными и анализом
import pandas as pd  # Для чтения и манипуляции данными в табличном формате
import numpy as np  # Для числовых операций и массивов
import statsmodels.api as sm  # Для построения моделей линейной регрессии и статистических тестов
from statsmodels.stats.outliers_influence import variance_inflation_factor  # Для расчета VIF (меры мультиколлинеарности)
from scipy import stats  # Для статистических функций, таких как skewness, kurtosis и t-тест
from sklearn.metrics import mean_squared_error  # Для расчета MSE (среднеквадратичной ошибки)
from google.colab import files  # Для загрузки файлов в Google Colab

# Шаг 1: Загрузка датасета из Excel файла (пользователь загружает файл 'for_G.xlsx')
uploaded = files.upload()  # Открывает диалог для загрузки файла; предполагаем, что файл называется 'for_G.xlsx'
file_name = list(uploaded.keys())[0]  # Получаем имя загруженного файла (первый ключ в словаре uploaded)
data = pd.read_excel(file_name, header=1)  # Читаем данные из Excel файла в DataFrame pandas, используя вторую строку как заголовок (header=1, так как row1 - X1.., row2 - имена колонок)

# Обработка missing values: удаляем строки с пропущенными значениями в количественных колонках (хотя в датасете их может не быть)
data = data.dropna()  # Удаляем все строки, содержащие NaN (пропущенные значения), чтобы соответствовать требованию задания

# Выбор колонок: количественные колонки из датасета
# Зависимая переменная y - 'Y' (количественная, представляющая уровень стресса: 1-Low, 2-Medium, 3-High, но трактуем как числовую для регрессии)
# Факторы x - остальные количественные: 'Age', 'Coffee_Intake', 'Caffeine_mg', 'Sleep_Hours', 'BMI (индекс массы тела)', 'Heart_Rate', 'Physical_Activity_Hours'
# Игнорируем 'Stress_Level' как категориальную (строковую)
y_column = 'Y'  # Название колонки с зависимой переменной y
x_columns = ['Age', 'Coffee_Intake', 'Caffeine_mg', 'Sleep_Hours', 'BMI (индекс массы тела)', 'Heart_Rate', 'Physical_Activity_Hours']  # Список колонок с факторами x
# Извлекаем y и X из DataFrame
y = data[y_column]  # Выбираем серию с зависимой переменной y
X = data[x_columns]  # Выбираем DataFrame с независимыми переменными (факторами)

# Добавляем константу для intercept (b в модели y = a_n x_n + ... + b)
X = sm.add_constant(X)  # Добавляем столбец с единицами для оценки константы b в регрессии

# Шаг 2: Построение первой версии линейной регрессии
model1 = sm.OLS(y, X).fit()  # Строим модель OLS (ordinaire least squares) и фитим на данных

# Вычисление коэффициента детерминации (R²)
r_squared1 = model1.rsquared  # Получаем R² из summary модели

# Вычисление MSE (Mean Squared Error)
y_pred1 = model1.predict(X)  # Предсказываем y на основе модели
mse1 = mean_squared_error(y, y_pred1)  # Рассчитываем MSE между реальными и предсказанными значениями

# Показатель системного эффекта факторов (предполагаем, что это F-статистика модели, как общая значимость факторов)
f_stat1 = model1.fvalue  # F-статистика для общей значимости модели

# Мера мультиколлинеарности: рассчитываем VIF для каждого фактора
vif_data1 = pd.DataFrame()  # Создаем пустой DataFrame для хранения VIF
vif_data1["feature"] = X.columns[1:]  # Добавляем названия факторов (исключая const)
vif_data1["VIF"] = [variance_inflation_factor(X.values, i+1) for i in range(len(X.columns[1:]))]  # Рассчитываем VIF для каждого фактора (i+1 из-за const)

# Вывод результатов первой модели (для отчета: коэффициенты в model1.summary(), показатели)
print("Первая модель:")  # Печатаем заголовок для результатов
print(model1.summary())  # Печатаем полный summary модели (коэффициенты, p-values и т.д.)
print(f"R²: {r_squared1}")  # Печатаем R²
print(f"MSE: {mse1}")  # Печатаем MSE
print(f"Показатель системного эффекта (F-статистика): {f_stat1}")  # Печатаем F-статистику
print("VIF (мера мультиколлинеарности):")  # Печатаем заголовок для VIF
print(vif_data1)  # Печатаем таблицу VIF

# Шаг 3: Построение матрицы корреляций (включая y и все факторы)
corr_matrix = data[[y_column] + x_columns].corr()  # Строим корреляционную матрицу для y и всех x
print("Матрица корреляций:")  # Печатаем заголовок
print(corr_matrix)  # Печатаем матрицу

# Отбор значимых факторов: на основе p-values из модели (<0.05) и корреляций с y (>0.1 по модулю, пример; адаптируйте по данным, чтобы осталось >=2)
# Также статистика значимости: p-values уже в summary, сравниваем с критическим (0.05)
significant_factors = []  # Создаем список для значимых факторов
for i, p in enumerate(model1.pvalues[1:]):  # Перебираем p-values факторов (исключая const)
    corr_with_y = corr_matrix[y_column].iloc[i+1]  # Корреляция фактора с y (iloc[i+1] из-за y первой)
    if p < 0.05 and abs(corr_with_y) > 0.1:  # Если p<0.05 и корреляция >0.1 по модулю (пример порога; выберите так, чтобы >=2 факторов)
        significant_factors.append(x_columns[i])  # Добавляем фактор в список
# Убеждаемся, что минимум 2 фактора; если меньше, берем топ-2 по значимости (например, lowest p-values)
if len(significant_factors) < 2:  # Проверяем количество отобранных факторов
    # Сортируем по p-values и берем топ-2
    pvalues_df = pd.DataFrame({'feature': x_columns, 'pvalue': model1.pvalues[1:]})  # DataFrame с p-values
    pvalues_df = pvalues_df.sort_values('pvalue')  # Сортируем по возрастанию p
    significant_factors = pvalues_df['feature'].head(2).tolist()  # Берем первые 2
print(f"Отобранные значимые факторы (на основе p<0.05 и |corr|>0.1, минимум 2): {significant_factors}")  # Печатаем отобранные факторы
print("Обоснование: Факторы выбраны по статистической значимости коэффициентов (p-value < 0.05) и корреляции с y (>0.1 по модулю).")  # Обоснование для отчета

# Шаг 4: Построение новой регрессии с отобранными факторами
X_selected = data[significant_factors]  # Выбираем только отобранные факторы
X_selected = sm.add_constant(X_selected)  # Добавляем константу
model2 = sm.OLS(y, X_selected).fit()  # Строим и фитим новую модель

# Вычисление метрик для второй модели (аналогично первой)
r_squared2 = model2.rsquared  # R² второй модели
y_pred2 = model2.predict(X_selected)  # Предсказания
mse2 = mean_squared_error(y, y_pred2)  # MSE
f_stat2 = model2.fvalue  # F-статистика
vif_data2 = pd.DataFrame()  # DataFrame для VIF
vif_data2["feature"] = X_selected.columns[1:]  # Названия факторов
vif_data2["VIF"] = [variance_inflation_factor(X_selected.values, i+1) for i in range(len(X_selected.columns[1:]))]  # VIF

# Вывод результатов второй модели и сравнение (для интерпретации: сравните R², MSE и т.д.; например, если R² близко, а MSE ниже - модель проще и лучше)
print("Вторая модель:")  # Заголовок
print(model2.summary())  # Summary
print(f"R²: {r_squared2} (vs первая: {r_squared1})")  # Сравнение R²
print(f"MSE: {mse2} (vs первая: {mse1})")  # Сравнение MSE
print(f"Показатель системного эффекта (F-статистика): {f_stat2} (vs первая: {f_stat1})")  # Сравнение F
print("VIF:")  # Заголовок VIF
print(vif_data2)  # Таблица VIF
print("Интерпретация: Сравните показатели - вторая модель использует только значимые факторы, что может снизить мультиколлинеарность (VIF) и улучшить интерпретируемость без сильной потери в R² или росте MSE.")  # Пример интерпретации для отчета

# Шаг 5: Критерий Фишера для целесообразности исключения факторов (F-test между полной и редуцированной моделями)
f_test = model1.compare_f_test(model2)  # Сравниваем модели: F-статистика, p-value, degrees of freedom (если p>0.05, исключение целесообразно - модель не ухудшилась значительно)
print(f"Результаты по критерию Фишера: F={f_test[0]}, p-value={f_test[1]}, df={f_test[2]}")  # Печатаем результаты
if f_test[1] > 0.05:  # Если p>0.05, различие не значимо
    print("Исключение факторов целесообразно (p>0.05, модель не ухудшилась значительно).")  # Вывод для отчета
else:  # Иначе
    print("Исключение факторов нецелесообразно (p<0.05, модель значимо ухудшилась).")  # Вывод для отчета

# Шаг 6: Проверка условий Гаусса-Маркова на второй (финальной) модели
residuals = model2.resid  # Получаем остатки (residuals) из модели

# 6.1: Случайность остатков - критерий поворотных точек (runs test для случайности)
def runs_test(res):  # Функция для runs test (количество runs/поворотов)
    median = np.median(res)  # Медиана остатков
    signs = np.sign(res - median)  # Знаки отклонений от медианы (полож/отриц)
    signs = signs[signs != 0]  # Удаляем нули, если есть
    runs = np.sum(np.diff(signs) != 0) + 1  # Количество runs (смен знаков +1)
    n = len(signs)  # Длина последовательности
    n_pos = np.sum(signs > 0)  # Количество положительных
    n_neg = np.sum(signs < 0)  # Количество отрицательных
    exp_runs = (2 * n_pos * n_neg / n) + 1  # Ожидаемое количество runs под H0 (случайность)
    var_runs = (2 * n_pos * n_neg * (2 * n_pos * n_neg - n)) / (n**2 * (n - 1))  # Дисперсия runs
    z = (runs - exp_runs) / np.sqrt(var_runs) if var_runs > 0 else 0  # Z-статистика для теста
    p_val = 2 * (1 - stats.norm.cdf(abs(z)))  # p-value (двусторонний нормальный тест)
    return runs, exp_runs, z, p_val  # Возвращаем runs, expected, z, p
runs, exp_runs, z, p_val = runs_test(residuals)  # Выполняем тест
print(f"Критерий поворотных точек (runs test): Observed runs={runs}, Expected={exp_runs}, Z={z}, p-value={p_val}")  # Печатаем
if p_val > 0.05:  # Если p>0.05
    print("Остатки случайны (H0 не отвергается).")  # Вывод
else:  # Иначе
    print("Остатки не случайны (H0 отвергается).")  # Вывод

# Коэффициенты асимметрии (skewness) и эксцесса (kurtosis); при необходимости RS-критерий (здесь используем Shapiro-Wilk для нормальности как альтернативу)
skew = stats.skew(residuals)  # Рассчитываем skewness (асимметрия; ~0 для нормальности)
kurt = stats.kurtosis(residuals)  # Рассчитываем excess kurtosis (эксцесс; ~0 для нормальности)
shapiro_stat, shapiro_p = stats.shapiro(residuals)  # Shapiro-Wilk тест на нормальность (RS? возможно Rank Sum, но здесь SW как для нормальности)
print(f"Асимметрия (skewness): {skew}, Эксцесс (kurtosis): {kurt}")  # Печатаем
print(f"Shapiro-Wilk (на нормальность, при необходимости как RS-альтернатива): stat={shapiro_stat}, p={shapiro_p}")  # Печатаем
if shapiro_p > 0.05:  # Если p>0.05
    print("Остатки нормально распределены (или близко; H0 не отвергается).")  # Вывод
else:  # Иначе
    print("Остатки не нормально распределены.")  # Вывод

# 6.2: Независимость остатков - критерий Дарбина-Уотсона (или альтернативный; здесь DW)
dw = sm.stats.durbin_watson(residuals)  # Рассчитываем Durbin-Watson statistic (~2 - нет автокорреляции; <1.5 положительная, >2.5 отрицательная)
print(f"Критерий Дарбина-Уотсона: {dw}")  # Печатаем
if 1.5 < dw < 2.5:  # Примерный диапазон для отсутствия автокорреляции
    print("Остатки независимы (DW близко к 2).")  # Вывод
else:  # Иначе
    print("Есть автокорреляция остатков.")  # Вывод

# 6.3: Равенство суммы остатков нулю (фактически среднее ~0; критерий Стьюдента на mean=0)
sum_res = np.sum(residuals)  # Сумма остатков (должна быть ~0)
mean_res = np.mean(residuals)  # Среднее остатков
t_stat, p_val = stats.ttest_1samp(residuals, 0)  # t-тест на mean=0 (Стьюдента)
print(f"Сумма остатков: {sum_res}, Среднее: {mean_res}")  # Печатаем
print(f"Критерий Стьюдента: t={t_stat}, p={p_val}")  # Печатаем результаты теста
if p_val > 0.05:  # Если p>0.05
    print("Среднее остатков не отличается от 0 (H0 не отвергается).")  # Вывод
else:  # Иначе
    print("Среднее остатков отличается от 0.")  # Вывод

# Завершение: анализ выполнен; используйте print-выводы для создания PDF-отчета (скопируйте значения, матрицы и т.д.)
# Для отчета также: ФИО/группа (вручную), цель (практика в линейной регрессии), датасет 'for_G.xlsx', колонки как выше, обработка - dropna если была.
print("Анализ завершен. Используйте выводы для отчета. Датасет: for_G.xlsx; Зависимая: Y; Факторы: {x_columns}; Отобранные: {significant_factors}.")  # Финальное сообщение

Saving for_G.xlsx to for_G (1).xlsx
Первая модель:
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     2427.
Date:                Sun, 28 Sep 2025   Prob (F-statistic):               0.00
Time:                        07:19:57   Log-Likelihood:                -5021.9
No. Observations:               10000   AIC:                         1.006e+04
Df Residuals:                    9992   BIC:                         1.012e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------

/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:579: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 10000.
  res = hypotest_fun_out(*samples, **kwds)
